# Image Processing SS 20 - Assignment - 05

### Deadline is 27.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage.data import chelsea
from skimage.color import rgb2gray

import random
from io import BytesIO
import cv2
from PIL import Image
import itertools

pylab.rcParams['figure.figsize'] = (12, 8)   # This makes the plot bigger

# Exercise 1 - Filters - 5 Points

Implement and apply the filters below to noisy images. Plot the noisy image vs the filtered one

In [ ]:
img = rgb2gray(chelsea() / 255.)
img.shape

In [ ]:
def salt_peper(img, p):
    peper_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    salt_mask = np.random.binomial(1, p, img.shape).astype(np.bool)
    img_noise = img.copy()
    img_noise[peper_mask] = 0
    img_noise[salt_mask] = 1
    return img_noise

salt_peper_noises = [0.01, 0.03, 0.1]
imgs_salt_peper = [salt_peper(img, p) for p in salt_peper_noises]
for img_salt_peper in imgs_salt_peper:
    plt.imshow(img_salt_peper, cmap='gray')
    plt.show()

In [ ]:
def gaussian_noise(img, std):
    return np.clip(img + np.random.normal(0, std, img.shape), 0, 1)

gaussian_noises = (0.05, 0.10, 0.2)
imgs_gaussian_noise = [gaussian_noise(img, s) for s in gaussian_noises]
for img_gaussian in imgs_gaussian_noise:
    plt.imshow(img_gaussian, cmap='gray')
    plt.show()

In [ ]:
def arithmetic_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(3, x - 3):
        for j in range(3, y - 3):
            val = 0
            for k in range(-3, 4):
                for l in range(-3, 4):
                    val += img[i + k, j + l]
            out[i, j] = val / (1/49)
    return out

def geometric_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x - 1):
        for j in range(1, y - 1):
            val = 1
            for k in range(-1, 2):
                for l in range(-1, 2):
                    val *= img[i + k, j + l]
            out[i, j] = val ** (1/9)
    return out

def harmonic_mean(img):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x - 1):
       for j in range(1, y - 1):
           val = 0
           for k in range(-1, 2):
               for l in range(-1, 2):
                   val += ( 1 / (img[i + k, j + l]) )
           out[i, j] = 9 / val
    return out

def contraharmoic_mean(img, q=-1):
    out = np.zeros_like(img)
    x, y = img.shape
    for i in range(1,x - 1):
        for j in range(1,y - 1):
            counter = 0
            denominator = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    counter += (img[i + k, j + l] ** (q + 1))
                    denominator += (img[i + k, j + l] ** (q))
            out[i, j] = counter / denominator
    return out


def sampling(x,i,j):
    result = np.zeros((7,7))
    for k in range(-3,4): 
        for l in range(-3,4): 
            result[k,l] = x[i+k,j+l] 
    return result


def adaptive_mean(img):
    out        = np.zeros_like(img)
    x, y       = img.shape
    var_global = np.std(img)
    for i in range(3,x - 3):
        for j in range(3,y - 3):
            window = sampling(img, i,j)
            var_local = np.std(window)
            m         = np.mean(window)
            out[i,j] = img[i,j] - (var_global / var_local) * (img[i,j] - m)
    return out

def adaptive_median(img):
    out        = np.zeros_like(img)
    x, y       = img.shape
    var_global = np.std(img)
    for i in range(3,x - 3):
        for j in range(3,y - 3):
            window = sampling(img, i,j)
            var_local = np.std(window)
            m         = np.median(window)
            out[i,j] = img[i,j] - (var_global / var_local) * (img[i,j] - m)
    return out


In [ ]:
filters = [arithmetic_mean, geometric_mean, harmonic_mean, adaptive_mean, contraharmoic_mean, adaptive_median]

In [ ]:

for filter in filters:
    for sp, img_sp in zip(salt_peper_noises, imgs_salt_peper):
        plt.suptitle(filter.__name__ + ", salt peper noise: {}".format(sp))
        plt.subplot(221)
        plt.imshow(img_sp, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_sp.copy()), cmap='gray')
        plt.show()


In [ ]:
for filter in filters:
    for gn, img_gn in zip(gaussian_noises, imgs_gaussian_noise):
        plt.suptitle(filter.__name__ + ", gaussian noise: {}".format(gn))
        plt.subplot(221)
        plt.imshow(img_gn, cmap='gray')
        plt.subplot(222)
        plt.imshow(filter(img_gn.copy()), cmap='gray')
        plt.show()

# Exercise 2 - Sharpening - 5 Points

Sharpen the image `img_blurry`. You need to implement the `sharp_laplacian` and `unsharp_masking` functions. You might
consider some processing steps before sharpening.

In [ ]:
from skimage.filters import gaussian

In [ ]:
img_blurry = gaussian_noise(gaussian(img , 1.6), 0.07)
plt.imshow(img_blurry, cmap='gray')

In [ ]:
def sharp_laplacian(img):
    """Perform sharpening with a laplacian filter"""
    w = np.array([ [0, 1, 0], [1, -4, 1], [0, 1, 0] ]) 
    x, y = img.shape
    laplacian_filter = np.zeros_like(img)

    for i in range(1, x - 1):
        for j in range(1, y - 1):
            pixel_calc = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    pixel_calc += w[k, l] * img[i - k, j - l]
                laplacian_filter[i, j] = pixel_calc   

    return  laplacian_filter


def unsharp_masking(img):
    """Perform sharpening by unsharp masking"""
    w = (1 / 9) * np.array([ [1,1,1], [1,1,1], [1,1,1] ])
    blurry_img = np.zeros_like(img)
    x, y = img.shape
    for i in range(1, x-1): 
        for j in range(1, y-1): 
            pixel_calc = 0
            for k in range(-1, 2):
                for l in range(-1, 2):
                    pixel_calc += w[k, l] * img[i - k, j - l]
            blurry_img[i, j] = pixel_calc
 
    return img - blurry_img

In [ ]:
img_filtered = arithmetic_mean(img_blurry) 
shapred_masking = sharp_laplacian(img_filtered)
plt.figure(figsize=(11,9))

plt.subplot(141), plt.imshow( img_blurry , cmap='gray'),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(142), plt.imshow( img_filtered, cmap='gray'),plt.title('Original + filter')
plt.xticks([]), plt.yticks([])
plt.subplot(143), plt.imshow(shapred_masking, cmap='gray'),plt.title('mask')
plt.xticks([]), plt.yticks([])
plt.subplot(144), plt.imshow(img_filtered + shapred_masking , cmap='gray'),plt.title('mask + (Original + filter)')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
img_filtered = arithmetic_mean(img_blurry) 
shapred_masking = unsharp_masking(img_filtered)
plt.figure(figsize=(11,9))

plt.subplot(141), plt.imshow( img_blurry , cmap='gray'),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(142), plt.imshow( img_filtered, cmap='gray'),plt.title('Original + filter')
plt.xticks([]), plt.yticks([])
plt.subplot(143), plt.imshow(shapred_masking, cmap='gray'),plt.title('mask')
plt.xticks([]), plt.yticks([])
plt.subplot(144), plt.imshow(img_filtered + shapred_masking , cmap='gray'),plt.title('mask + (Original + filter)')
plt.xticks([]), plt.yticks([])
plt.show()